# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [1]:
!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [3]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = os.getenv('ELASTIC_CLOUD_ID')

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = os.getenv('ELASTIC_ENCODED')

# https://platform.openai.com/api-keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="lab_multi_query",
    embedding=embeddings,
)

## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [4]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [9]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def metadata_func(record: dict, metadata: dict) -> dict:
    #Populate the metadata dictionary with keys name, summary, url, category, and updated_at.
    metadata['name'] = record['name']
    metadata['summary'] = record['summary']
    metadata['url'] = record['url']
    metadata['category'] = record['category']
    metadata['created_on'] = record['created_on']

    return metadata


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200 #define chunk size and chunk overlap
)
docs = loader.load_and_split(text_splitter=text_splitter)

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [10]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name='lab_multi_query',
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [11]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible. 
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)


def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The NASA sales team is a part of the Americas region in the sales organization of the company. It is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. The team is responsible for promoting and selling the company's products and services in the North and South American markets. They work closely with other departments, such as marketing, product development, and customer support, to ensure the company's success in these regions.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [12]:
print(chain.invoke("What is the Wfh Policy?"))

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the Work from Home (Wfh) Policy?', '2. How does the Wfh Policy impact employees?', '3. What are the guidelines and regulations of the Wfh Policy?']



The WFH (Work From Home) Policy is a set of guidelines and support provided by the company for employees to work remotely. It was implemented in March 2020 to ensure the continuity and productivity of business operations during the COVID-19 pandemic and beyond. The policy applies to all eligible employees and includes provisions for equipment and resources, workspace, communication, work hours and availability, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, policy review and updates, and questions and concerns. The policy was updated in May 2023 and May 2022 to include a hybrid work model, with employees required to work from the office three days a week and two days designated for remote work.


In [13]:
print(chain.invoke("Can you tell me the most important information of the pet Policy?"))

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the key details of the pet Policy?', '2. Can you provide me with the most crucial information regarding the pet Policy?', '3. What is the most significant information that I should know about the pet Policy?']



The most important information of the pet policy is that employees must obtain prior approval from their supervisor and the HR department before bringing their pets to the office. The approval process includes submitting a written request, providing proof of vaccinations and any required licenses, and obtaining written consent from all employees who share the workspace with the pet owner. Employees are responsible for the behavior and well-being of their pets while in the office and must closely supervise them. Employees with allergies or phobias related to pets must inform the HR department, and the company reserves the right to deny or revoke pet approval. Pets are not allowed in certain areas of the office, and employees are responsible for maintaining a clean and hygienic work environment. Pet owners are liable for any damage or injury caused by their pets, and pets must be kept on a leash or in a secure enclosure when not in the employee's immediate work area. The pet policy will

In [14]:
print(chain.invoke("Am a able to bring my cat to the office?"))

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can I bring my cat to the office?', '2. Is it possible for me to bring my cat with me to work?', '3. Would it be acceptable for me to bring my feline companion to the office?']


According to the office pet policy, employees are allowed to bring their pets to the office, including cats, subject to approval by the HR department. The approval process includes submitting a written request, providing proof of vaccinations and licenses, and obtaining written consent from all employees who share the workspace with the pet owner. However, the HR department reserves the right to deny or revoke pet approval based on specific circumstances or concerns raised by other employees. It is important to follow the guidelines and procedures outlined in the policy to ensure a positive and inclusive work environment for all employees, visitors, and pets. If you have any further questions or concerns, please direct them to your supervisor or the HR department.


In [15]:
print(chain.invoke("What day is today?"))

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the current date?', '2. Can you tell me the day of the week?', '3. What is the date today?']


I am an AI and do not have access to real-time information. I am unable to answer this question.


In [16]:
print(chain.invoke("Can you give me the main topics of the documents?"))

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the primary subjects covered in the documents?', '2. Could you provide me with the key themes of the documents?', '3. Can you list the main topics discussed in the documents?']



1. New Employee Onboarding Guide
2. Office Pet Policy
3. Fy2024 Company Sales Strategy
4. Updating Your Tax Elections Forms
5. Intellectual Property Policy
6. Code of Conduct
7. WFH Policy Update May 2023


In [17]:
print(chain.invoke("How many words contains the 'New Employee Onboarding Guide'?"))

INFO:langchain.retrievers.multi_query:Generated queries: ["1. What is the word count of the 'New Employee Onboarding Guide'?", "2. Can you provide the number of words in the 'New Employee Onboarding Guide'?", "3. How many total words are included in the 'New Employee Onboarding Guide'?"]



The 'New Employee Onboarding Guide' contains approximately 500 words.
